# 뉴스기사 분류모델

In [ ]:
# 코랩 형태소 분석 엔진 설치
#!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
#!bash Mecab-ko-for-Google-Colab/install_mecab-ko_on_colab_light_220429.sh

In [1]:
import numpy as np
import seaborn as sb
import requests

from pandas import DataFrame
from pandas import read_excel
#from konlpy.tag import Mecab
from matplotlib import pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, GRU
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.utils import to_categorical

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

뉴스기사 데이터셋

In [2]:
origin = read_excel('./news.xlsx')
origin.head()

,category,content
0,IT_과학,"“갤럭시S9 20만 원대, 아이폰6S 0원!” 모비톡, 가정의 달 이벤트 '갤럭시노..."
1,IT_과학,'모비톡' 단독으로 진행되는 5월 가정의 달 이벤트에 이용자들의 폭발적인 반응이 나...
2,IT_과학,"이 밖에도 '모비톡'은 '갤럭시노트8', 'V30' 구매 시 '닌텐도 스위치'를 증..."
3,IT_과학,"LG 그램’, 100만대 판매기념 한정판 나왔다 LG전자가 ‘그램’ 노트북 누적판매..."
4,IT_과학,LG전자는 베스트샵 등 주요 매장에서 23일부터 그램 한정판 500대를 판매한다고 ...


In [3]:
# 불용어 
r = requests.get("https://data.hossam.kr/korean_stopwords.txt")
r.encoding = 'utf-8'
stopwords = r.text.split('\n')
stopwords

['이',
 '있',
 '하',
 '것',
 '들',
 '그',
 '되',
 '수',
 '이',
 '보',
 '않',
 '없',
 '나',
 '사람',
 '주',
 '아니',
 '등',
 '같',
 '우리',
 '때',
 '년',
 '가',
 '한',
 '지',
 '대하',
 '오',
 '말',
 '일',
 '그렇',
 '위하',
 '때문',
 '그것',
 '두',
 '말하',
 '알',
 '그러나',
 '받',
 '못하',
 '일',
 '그런',
 '또',
 '문제',
 '더',
 '사회',
 '많',
 '그리고',
 '좋',
 '크',
 '따르',
 '중',
 '나오',
 '가지',
 '씨',
 '시키',
 '만들',
 '지금',
 '생각하',
 '그러',
 '속',
 '하나',
 '집',
 '살',
 '모르',
 '적',
 '월',
 '데',
 '자신',
 '안',
 '어떤',
 '내',
 '내',
 '경우',
 '명',
 '생각',
 '시간',
 '그녀',
 '다시',
 '이런',
 '앞',
 '보이',
 '번',
 '나',
 '다른',
 '어떻',
 '여자',
 '개',
 '전',
 '들',
 '사실',
 '이렇',
 '점',
 '싶',
 '말',
 '정도',
 '좀',
 '원',
 '잘',
 '통하',
 '소리',
 '놓',
 '!',
 '"',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '...',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '\\',
 '^',
 '_',
 '`',
 '|',
 '~',
 '·',
 '—',
 '——',
 '‘',
 '’',
 '“',
 '”',
 '…',
 '、',
 '。',
 '〈',
 '〉',
 '《',
 '》',
 '가',
 '가까스로',
 '가령',
 '각',
 '각각',
 '각자',
 '각종',
 '

### 데이터 전처리

In [4]:
# 뉴스기사에서 영어, 특수문자를 제거하고 한글만 남기기 

df = origin.copy()

df['content'] = df['content'].str.replace("[^ㄱ-하-ㅣ-가-힣]","",regex=True)
df['content'].replace("",np.nan,inplace=True)
print('데이터 크기:' ,df['content'].shape)
print('결측치 크기: ',df['content'].isna().sum())


데이터 크기: (74273,)
결측치 크기:  368


In [ ]:
# 결측치 제거
df.dropna(inplace=True)
print('결측치크기 :',df['content'].isna().sum())

결측치크기 : 0


In [7]:
# 종속변수 값의 종류
category =  list(df['category'].unique())
category

['IT_과학', '경제', '사회', '생활_문화', '세계', '정치']

In [9]:
# 종속변수 값의 종류를 딕셔너리로 변환
cate_dict = {}
for i, v in enumerate(category) :
    cate_dict[v] = i
cate_dict

{'IT_과학': 0, '경제': 1, '사회': 2, '생활_문화': 3, '세계': 4, '정치': 5}

In [10]:
# 종속변수 라벨링
df['category'] = df['category'].map(cate_dict)
df['category'].unique()

array([0, 1, 2, 3, 4, 5], dtype=int64)